In [2]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import os
import cv2
from imutils.video import VideoStream
import imutils

In [3]:
def detect_and_predict_mask(frame,faceNet,maskNet):
    (h,w)=frame.shape[:2]
    blob=cv2.dnn.blobFromImage(frame,1.0,(300,300),(104.0,177.0,123.0))
    faceNet.setInput(blob)
    detections=faceNet.forward()
    
    faces=[]
    locs=[]
    preds=[]
    
    for i in range(0,detections.shape[2]):
        confidence=detections[0,0,i,2]

        if confidence>0.5:
                box=detections[0,0,i,3:7]*np.array([w,h,w,h])
                (startx,starty,endx,endy)=box.astype('int')
                (startx,starty)=(max(0,startx),max(0,starty))
                (endx,endy)=(min(w-1,endx),min(h-1,endy))

                face=frame[starty:endy, startx:endx]
                face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
                face=cv2.resize(face,(224,224))
                face=img_to_array(face)
                face=preprocess_input(face)

                faces.append(face)
                locs.append((startx,starty,endx,endy))

        if len(faces)>0:
            faces=np.array(faces,dtype='float32')
            preds=maskNet.predict(faces,batch_size=12)
            
        return (locs,preds)

In [4]:
prototxtPath=os.path.sep.join([r'C:\Users\Yashwanth Brahma\Documents\Mask-Detection\face-detectors','deploy.prototxt'])
weightsPath=os.path.sep.join([r'C:\Users\Yashwanth Brahma\Documents\Mask-Detection\face-detectors','res10_300x300_ssd_iter_140000.caffemodel'])

In [5]:
faceNet=cv2.dnn.readNet(prototxtPath,weightsPath)

In [6]:
maskNet=load_model(r'C:\Users\Yashwanth Brahma\Documents\Mask-Detection\mobilenet_v2.model')

In [11]:
vs=VideoStream(src=0).start()

while True:
    frame=vs.read()
    frame=imutils.resize(frame,width=400)
    
    (locs,preds)=detect_and_predict_mask(frame,faceNet,maskNet)
    
    for (box,pred) in zip(locs,preds):
        (startx,starty,endx,endy)=box
        (mask,withoutMask)=pred
        
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)

        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)

        cv2.putText(frame,label,(startx,starty-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(frame,(startx,starty),(endx,endy),color,2)

    cv2.imshow("Frame",frame)
    key=cv2.waitKey(1) & 0xFF
    
    if key==ord('q'):
        break
        

cv2.destroyAllWindows()
vs.stop()
del(vs)
